# <font color='darkblue'>Import Important Libraries</font>

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

# <font color='darkblue'>Read The Data</font>

In [3]:
NewsData = pd.read_json('./husna.json')

In [4]:
NewsData.drop(['publisher', 'published_at', 'article_type', 'crawled_at', 'summary', '_id'], axis=1, inplace=True)

# <font color='darkblue'>Remve Null</font>

In [5]:
NewsData.content = NewsData.content.apply(lambda x: np.nan if len(x)==0 else x)
NewsData.title = NewsData.title.apply(lambda x: np.nan if len(x)==0 else x)
NewsData.tags = NewsData.tags.apply(lambda x: np.nan if len(x)==0 else x)
NewsData.url = NewsData.url.apply(lambda x: np.nan if len(x)==0 else x)

In [6]:
NewsData.dropna(inplace=True)

In [7]:
NewsData.content[3]

['كشفت أمين عام وزارة التربية والتعليم للشؤون المالية والإدارية الدكتورة نجوى القبيلات بأن مجلس الوزراء طلب منا السيناريوهات المقترحة في حال تثبيت التوقيت الصيفي طوال العام، وقدمت الوزارة تلك السيناريوهات.',
 'وشرحت القبيلات لـ  حسنى  اليوم طبيعة الدوام خلال الفترة المقبلة، بعد قرار مجلس الوزراء يوم أمس تثبيت التوقيت الصيفي طوال العام، حيث يبدأ دوام المدارس بالتوقيت الجديد يوم الأحد الموافق 30/10/2022، وينتهي يوم الأحد الموافق 2/4/2022.',
 'يبدأ الطابور الصباحي في مدارس الفترة الواحدة والبالغ نسبتها 80%، الساعة 8:15، وتبدأ الحصّة الأولى الساعة 8:30، ومدّة الحصة 45 دقيقة، وينتهي الدوام بانتهاء الحصص حسب الصفوف، وفقاً للقبيلات.',
 'فيما يبدأ الطابور الصباحي في مدارس الفترتين، الساعة 8:15، وتبدأ الحصّة الأولى الساعة 8:30، وتم تخفيض الحصّة لتصبح 35 دقيقة فقط، وينتهي دوام الفترة الأولى الساعة 1:10، وينتهي الدوام في الفترة الثانية المسائية الساعة 5:20 قبل غروب الشمس، وسيتم فتح أبواب المدارس الساعة 7:45 لاستقبال الطلبة، علما بأن عدد المدارس على نظام الفترتين الدراسيتين، (882) مدرسة.',
 'فيما 

# <font color='darkblue'>Remove Punctuation</font>

In [8]:
Punctuations = '[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~،٪\\’’؟.]'

In [9]:
def remove_punctuations(text):
    text = str(text)
    for punctuation in Punctuations:
        text = text.replace(punctuation, ' ')
    return text

In [10]:
NewsData["content"] = NewsData["content"].apply(lambda x: remove_punctuations(x))

In [11]:
NewsData['content'][3]

'  كشفت أمين عام وزارة التربية والتعليم للشؤون المالية والإدارية الدكتورة نجوى القبيلات بأن مجلس الوزراء طلب منا السيناريوهات المقترحة في حال تثبيت التوقيت الصيفي طوال العام  وقدمت الوزارة تلك السيناريوهات     وشرحت القبيلات لـ  حسنى  اليوم طبيعة الدوام خلال الفترة المقبلة  بعد قرار مجلس الوزراء يوم أمس تثبيت التوقيت الصيفي طوال العام  حيث يبدأ دوام المدارس بالتوقيت الجديد يوم الأحد الموافق 30 10 2022  وينتهي يوم الأحد الموافق 2 4 2022     يبدأ الطابور الصباحي في مدارس الفترة الواحدة والبالغ نسبتها 80   الساعة 8 15  وتبدأ الحصّة الأولى الساعة 8 30  ومدّة الحصة 45 دقيقة  وينتهي الدوام بانتهاء الحصص حسب الصفوف  وفقاً للقبيلات     فيما يبدأ الطابور الصباحي في مدارس الفترتين  الساعة 8 15  وتبدأ الحصّة الأولى الساعة 8 30  وتم تخفيض الحصّة لتصبح 35 دقيقة فقط  وينتهي دوام الفترة الأولى الساعة 1 10  وينتهي الدوام في الفترة الثانية المسائية الساعة 5 20 قبل غروب الشمس  وسيتم فتح أبواب المدارس الساعة 7 45 لاستقبال الطلبة  علما بأن عدد المدارس على نظام الفترتين الدراسيتين   882  مدرسة     فيما نوه

# <font color='darkblue'>Remove Non Arabic</font>

In [12]:
Arabic = open("./Arabic.txt", "r")
ArabicLetters = Arabic.read()
ArabicLetters = ArabicLetters.replace('\n', ' ')

In [13]:
def remove_non_arabic(data):
    arabic_letter = ""
    for word in data:
        if word in ArabicLetters :
            arabic_letter = arabic_letter +word 
    return arabic_letter

In [14]:
NewsData['content'] = NewsData['content'].apply(lambda x:remove_non_arabic(x))

In [15]:
NewsData['content'][3]

'  كشفت أمين عام وزارة التربية والتعليم للشؤون المالية والإدارية الدكتورة نجوى القبيلات بأن مجلس الوزراء طلب منا السيناريوهات المقترحة في حال تثبيت التوقيت الصيفي طوال العام  وقدمت الوزارة تلك السيناريوهات     وشرحت القبيلات لـ  حسنى  اليوم طبيعة الدوام خلال الفترة المقبلة  بعد قرار مجلس الوزراء يوم أمس تثبيت التوقيت الصيفي طوال العام  حيث يبدأ دوام المدارس بالتوقيت الجديد يوم الأحد الموافق     وينتهي يوم الأحد الموافق        يبدأ الطابور الصباحي في مدارس الفترة الواحدة والبالغ نسبتها    الساعة    وتبدأ الحصة الأولى الساعة    ومدة الحصة  دقيقة  وينتهي الدوام بانتهاء الحصص حسب الصفوف  وفقا للقبيلات     فيما يبدأ الطابور الصباحي في مدارس الفترتين  الساعة    وتبدأ الحصة الأولى الساعة    وتم تخفيض الحصة لتصبح  دقيقة فقط  وينتهي دوام الفترة الأولى الساعة    وينتهي الدوام في الفترة الثانية المسائية الساعة   قبل غروب الشمس  وسيتم فتح أبواب المدارس الساعة   لاستقبال الطلبة  علما بأن عدد المدارس على نظام الفترتين الدراسيتين     مدرسة     فيما نوهت القبيلات إلى وجود فترة امتحانات في شهر كانون ال

# <font color='darkblue'>Remove Unnecessary Spaces</font> 

In [16]:
def removeUnnecessarySpaces(text):
    No_Unnecessary_Spaces = re.sub(r'[\n\t\s ]+', ' ', text)
    return No_Unnecessary_Spaces

In [17]:
NewsData['content'] = NewsData['content'].apply(lambda x:removeUnnecessarySpaces(x))

In [18]:
NewsData['content'][3]

' كشفت أمين عام وزارة التربية والتعليم للشؤون المالية والإدارية الدكتورة نجوى القبيلات بأن مجلس الوزراء طلب منا السيناريوهات المقترحة في حال تثبيت التوقيت الصيفي طوال العام وقدمت الوزارة تلك السيناريوهات وشرحت القبيلات لـ حسنى اليوم طبيعة الدوام خلال الفترة المقبلة بعد قرار مجلس الوزراء يوم أمس تثبيت التوقيت الصيفي طوال العام حيث يبدأ دوام المدارس بالتوقيت الجديد يوم الأحد الموافق وينتهي يوم الأحد الموافق يبدأ الطابور الصباحي في مدارس الفترة الواحدة والبالغ نسبتها الساعة وتبدأ الحصة الأولى الساعة ومدة الحصة دقيقة وينتهي الدوام بانتهاء الحصص حسب الصفوف وفقا للقبيلات فيما يبدأ الطابور الصباحي في مدارس الفترتين الساعة وتبدأ الحصة الأولى الساعة وتم تخفيض الحصة لتصبح دقيقة فقط وينتهي دوام الفترة الأولى الساعة وينتهي الدوام في الفترة الثانية المسائية الساعة قبل غروب الشمس وسيتم فتح أبواب المدارس الساعة لاستقبال الطلبة علما بأن عدد المدارس على نظام الفترتين الدراسيتين مدرسة فيما نوهت القبيلات إلى وجود فترة امتحانات في شهر كانون الأول ديسمبر المقبل حيث يبدأ الامتحان الساعة التاسعة أو العاشرة و

In [19]:
NewsData = NewsData.reset_index()


# <font color='darkblue'>Word Tokenizer</font> 

In [20]:
NewsData['Word tokenize'] = NewsData.content.apply(word_tokenize)

# <font color='darkblue'>Remove Stopwords</font> 

In [21]:
StopWord = open("./StopWords.txt", "r")
StopWords = StopWord.read()
StopWords = StopWords.replace('\n', ' ')


In [22]:
def remove_stop_words_stem_words(data):
       file_clean_k =pd.DataFrame()
       for index,entry in enumerate(data):
           Final_words = []
           stemmer = SnowballStemmer('arabic')
           for word, tag in pos_tag(entry):
               if word not in StopWords:
                   word_Final = stemmer.stem(word)
                   Final_words.append(word_Final)
                   file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
       return file_clean_k

In [23]:
df_clean = remove_stop_words_stem_words(NewsData['Word tokenize']) 

In [24]:
df_clean=df_clean.replace(to_replace ="\[.", value = '', regex = True)
df_clean=df_clean.replace(to_replace ="'", value = '', regex = True)
df_clean=df_clean.replace(to_replace =" ", value = '', regex = True)
df_clean=df_clean.replace(to_replace ='\]', value = '', regex = True)

In [25]:
df_clean = df_clean.reset_index()


In [26]:
NewsData.insert(loc=3, column='Clean_Keyword', value=df_clean['Keyword_final'].tolist())


In [27]:
NewsData['Clean_Keyword'][3]

'كشف,ام,ترب,تعليم,للشء,مال,ادار,دكتور,نجوى,قبيل,طلب,سيناريوه,مقترح,تثب,توق,صيف,طوال,عام,قدم,وزار,سيناريوه,شرح,قبيل,حسنى,طبيع,دوام,فتر,مقبل,تثب,توق,صيف,طوال,عام,يبدء,دوام,مدارس,توق,جديد,احد,موافق,وين,احد,موافق,يبدء,طابور,صباح,مدارس,فتر,واحد,الغ,نسب,تبدء,الحص,اولى,ومد,الحص,دقيق,وين,دوام,انتهاء,حصص,صفوف,وفق,قبيل,يبدء,طابور,صباح,مدارس,فترت,تبدء,الحص,اولى,وتم,تخفيض,الحص,تصبح,دقيق,وين,دوام,فتر,اولى,وين,دوام,فتر,مساء,غروب,شمس,سيتم,ابواب,مدارس,استقبال,طلب,علم,مدارس,نظام,فترت,دراسيت,مدرس,نوه,قبيل,جود,فتر,امتحان,شهر,اول,يبدء,امتح,تاسع,عاشر,لمد,ساع,وذل,دوام,فتر,سيك,اغلب,طلب,منازل,بوق,ظهير,تبدء,عطل,شتو,تمتد,جلس,عقد,اربعاء,تثب,ب,توق,صيف,طوال,عام,جاء,قرار,دراس,مستفيض,اجر,حكوم,وقوف,جدوى,تثب,توق,صيف,وفق,جاء,قرار,ادى,تعديل,دوام,مدارس,فتر,صباح,والفت,طلب,وزا,ترب,تعليم,تعليم,عال,تعديل,دوام,يتناسب,مواعيد,شروق,شمس'

# <font color='darkblue'>TFIDF</font> 

In [28]:
## Create Vocabulary
voc = set()

for doc in NewsData.Clean_Keyword:
    voc.update(doc.split(','))

voc = list(voc)

# Intializating the tfIdf model
tfidf = TfidfVectorizer(vocabulary=voc)

# Fit the TfIdf model
tfidf.fit(NewsData.Clean_Keyword)

# Transform the TfIdf model
tfidf_tran=tfidf.transform(NewsData.Clean_Keyword)


In [29]:
def gen_vector_T(tokens):

    Q = np.zeros((len(voc)))
    
    x= tfidf.transform(tokens)
    for token in tokens[0].split(','):
        try:
            ind = voc.index(token)
            Q[ind]  = x[0, tfidf.vocabulary_[token]]
        except:
            pass
    return Q

# <font color='darkblue'>Cosine Similarity</font> 

In [30]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a) * np.linalg.norm(b))
    return cos_sim

In [31]:
def Search(k, query):
      preprocessed_query  = re.sub("\W+", " ", query).strip()
      tokens = word_tokenize((preprocessed_query))
      QueryData = pd.DataFrame(columns=['q_clean'])
      QueryData.loc[0,'q_clean'] =tokens
      QueryData['q_clean'] =remove_stop_words_stem_words(QueryData['q_clean'])
      QueryData=QueryData.replace(to_replace ="\[.", value = '', regex = True)
      QueryData=QueryData.replace(to_replace ="'", value = '', regex = True)
      QueryData=QueryData.replace(to_replace =" ", value = '', regex = True)
      QueryData=QueryData.replace(to_replace ='\]', value = '', regex = True)

      d_cosines = []
    
      query_vector = gen_vector_T(QueryData['q_clean'])
    
      for d in tfidf_tran.A:
        
        
          d_cosines.append(cosine_sim(query_vector, d))
                    
      out = np.array(d_cosines).argsort()[-k:][::-1]
      d_cosines.sort()
      a = pd.DataFrame()
      for i,index in enumerate(out):
          a.loc[i,'url'] = NewsData['url'][index]
          a.loc[i,'ُTitle'] = NewsData['title'][index]
          a.loc[i,'Index'] = str(index)
      for j,simScore in enumerate(d_cosines[-k:][::-1]):
          a.loc[j,'Score'] = simScore
      return a

In [38]:
%time Search(5,'اسعار الذهب')

CPU times: user 196 ms, sys: 93.3 ms, total: 289 ms
Wall time: 308 ms


,url,ُTitle,Index,Score
0,https://husna.fm/%D8%AD%D8%B3%D9%86%D9%89/%D9%...,استقرار أسعار الذهب في الأردن وازدياد الطلب عل...,3122,0.601359
1,https://husna.fm/%D8%AD%D8%B3%D9%86%D9%89/%D9%...,أسعار الذهب اليوم في الأردن ترتفع بحدود 40 قرش,3096,0.499780
2,https://husna.fm/%D8%AD%D8%B3%D9%86%D9%89/%D9%...,أسعار الذهب في الأردن اليوم الأربعاء,3113,0.449976
3,https://husna.fm/%D8%AD%D8%B3%D9%86%D9%89/%D9%...,انخفاض أسعار الذهب في الأردن الثلاثاء ما بين 1...,3111,0.449440
4,https://husna.fm/%D8%AD%D8%B3%D9%86%D9%89/%D8%...,أسعار الذهب في الأردن تنخفض ما بين 10 إلى 20 ف...,3073,0.434269
